## Set up

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-huggingface text-generation transformers bitsandbytes accelerate langchain bitsandbytes langchain_community

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        return_full_text=False,
    ),
    model_kwargs={"quantization_config": quantization_config},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Build the agent

In [ ]:
from typing import Literal
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode

memory = MemorySaver()  # Bộ lưu trữ trạng thái (chỉ thử nghiệm)


@tool
def search(query: str):
    """Gọi để lướt web."""  # Hàm giả lập
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)
model = ChatHuggingFace(llm=llm)  # llm
bound_model = model.bind_tools(tools)  # Liên kết với tools để llm có thể gọi khi cần


def should_continue(state: MessagesState):
    """Trả về node tiếp theo để thực thi."""
    last_message = state["messages"][-1]
    # Nếu không có gọi công cụ (tool calls), quá trình kết thúc
    if not last_message.tool_calls:
        return END
    # Nếu có thì tiếp tục
    return "action"


# Định nghĩa hàm để gọi model
def call_model(state: MessagesState):
    response = bound_model.invoke(state["messages"])
    # Chúng ta trả về một danh sách, vì danh sách này sẽ được thêm vào danh sách hiện có
    return {"messages": response}


# Định nghĩa một đồ thị mới
workflow = StateGraph(MessagesState)

# Định nghĩa luồng xử lý (agent-action-agent)
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Đặt điểm bắt đầu là `agent`
# Điều này có nghĩa là node này sẽ được gọi đầu tiên
workflow.add_edge(START, "agent")

# Thêm một cạnh có điều kiện
workflow.add_conditional_edges(
    # Đầu tiên, định nghĩa node bắt đầu. Chúng ta sử dụng `agent`.
    # Điều này có nghĩa là các cạnh này sẽ được thực hiện sau khi node `agent` được gọi.
    "agent",
    # Tiếp theo, truyền vào hàm sẽ xác định node nào được gọi tiếp theo.
    should_continue,
    # Tiếp theo, truyền vào bản đồ đường dẫn - tất cả các node có thể mà cạnh này có thể đi đến
    ["action", END],
)

# Thêm một cạnh bình thường từ `tools` đến `agent`.
# Điều này có nghĩa là sau khi `tools` được gọi, node `agent` sẽ được gọi tiếp.
workflow.add_edge("action", "agent")

# Cuối cùng, biên dịch nó!
# Điều này biên dịch đồ thị thành một LangChain Runnable,
# có nghĩa là bạn có thể sử dụng nó giống như bất kỳ runnable nào khác
app = workflow.compile(checkpointer=memory)

In [ ]:

from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello, Bob! It's great to meet you today. How may I assist you? Please let me know what you're looking for, and I'll do my best to provide you with the information you need.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to your personal information. My training data only goes up until a certain point, and I'm not connected to any external databases that could provide me with your name. However, if you can provide me with any other details, such as your birthdate or address, I might be able to help you narrow down your identity. But for now, I'm afraid I can't tell you your name. Is there anything else I can help you with?


## Filtering messages

In [ ]:
from typing import Literal

from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode

memory = MemorySaver()


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"


def filter_messages(messages: list):
    return messages[-2:]


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = filter_messages(state["messages"])
    response = bound_model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the pathmap - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# (because we set `messages[-1:]` in the filter messages argument)
input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello, Bob! It's great to meet you today. How may I assist you? Please let me know what you're looking for, and I'll do my best to provide you with the information you need.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I do not have information about your identity. Please provide me with your name, or let me know in what context you are asking this question, so I can better understand how to assist you.


## Summary

### Build the chatbot

In [ ]:
from typing import Literal
from langchain_core.messages import SystemMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

memory = MemorySaver()


# We will add a `summary` attribute (in addition to `messages` key,
# which MessagesState already has)
class State(MessagesState):
    summary: str




# Define the logic to call the model
def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# We now define the logic for determining whether to end or summarize the conversation
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END


def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

### Using the graph

In [ ]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello, Bob! It's great to meet you today. How may I assist you? Please let me know what you're looking for, and I'll do my best to provide you with the information you need.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to your personal information. My training data only goes up until a certain point, and I'm not connected to any external databases that could provide me with your name. However, if you can provide me with any other details, such as your birthdate or address, I might be able to help you narrow down your identity. But for now, I'm afraid I can't tell you your name. Is there anything else I can help you with?
=========================

In [ ]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='56385bdb-fee2-43d2-bece-8badefdfd728'),
  AIMessage(content="Hello, Bob! It's great to meet you today. How may I assist you? Please let me know what you're looking for, and I'll do my best to provide you with the information you need.", additional_kwargs={}, response_metadata={}, id='run-5707a9be-edaf-46d3-aed9-49867acde7ab-0'),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='b275ee62-3513-4558-aa23-44781fcbc9ab'),
  AIMessage(content="I'm sorry, but I don't have access to your personal information. My training data only goes up until a certain point, and I'm not connected to any external databases that could provide me with your name. However, if you can provide me with any other details, such as your birthdate or address, I might be able to help you narrow down your identity. But for now, I'm afraid I can't tell you your name. Is there anything e

In [ ]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
================================== Ai Message ==================================

Yes, the Boston Celtics have a rich history of success, and they've consistently been one of the top teams in the NBA. In fact, they've made it to the NBA Finals a total of 21 times, winning the championship 17 times. Some of their most recent championships came in 2008 and 2010, led by players like Paul Pierce, Kevin Garnett, and Ray Allen. The team's success is due in part to their strong coaching staff, including current head coach Brad Stevens, who has led the team to the playoffs in each of his first six seasons at the helm. If you're interested in learning more about the Celtics, I'd be happy to provide you with more information about their current roster, upcoming games, and team statistics. Just let me know!
================================ Remove Message ================================


==

In [ ]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like how much they win', additional_kwargs={}, response_metadata={}, id='556838ae-7715-4fbd-b616-014a9ec2b4cf'),
  AIMessage(content="Yes, the Boston Celtics have a rich history of success, and they've consistently been one of the top teams in the NBA. In fact, they've made it to the NBA Finals a total of 21 times, winning the championship 17 times. Some of their most recent championships came in 2008 and 2010, led by players like Paul Pierce, Kevin Garnett, and Ray Allen. The team's success is due in part to their strong coaching staff, including current head coach Brad Stevens, who has led the team to the playoffs in each of his first six seasons at the helm. If you're interested in learning more about the Celtics, I'd be happy to provide you with more information about their current roster, upcoming games, and team statistics. Just let me know!", additional_kwargs={}, response_metadata={}, id='run-ecadbe11-7520-4eac-b785-4314c43f5f31-0')],
 'sum

In [ ]:
input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I'm not able to determine your identity based on our previous conversation. When we spoke earlier, you introduced yourself as Bob, but you mentioned that you couldn't remember your name. If you're still having trouble remembering your name, I suggest trying to recall any other details about yourself, such as where you live, your occupation, or any notable experiences you've had recently. This might help jog your memory and narrow down your identity. Alternatively, you could try reaching out to friends, family members, or coworkers for assistance in identifying yourself. I hope this helps! Let me know if there's anything else I can do for you.


In [ ]:
input_message = HumanMessage(content="what NFL team do you think I like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


================================ Human Message =================================

what NFL team do you think I like?
================================== Ai Message ==================================

I'm sorry, but I don't have any information about your preferences regarding NFL teams. If you're asking me to make an educated guess based on your location or other factors, I'd say that if you're a fan of the Boston Celtics, you might also be a fan of the New England Patriots, as they're both professional sports teams based in the Boston area. However, I'd recommend checking your favorite sports teams list or looking at your social media profiles to see if you've mentioned any NFL teams you support. If you're still unsure, you could try reaching out to friends, family members, or coworkers for assistance in identifying your favorite NFL team. I hope this helps! Let me know if there's anything else I can do for you.


In [ ]:
input_message = HumanMessage(content="i like the patriots!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like the patriots!
================================== Ai Message ==================================

That's great to hear! The New England Patriots are a highly successful NFL team, having won six Super Bowl championships under the leadership of quarterback Tom Brady and head coach Bill Belichick. Some of their most recent championships came in 2016, 2017, and 2018. The team's success is due in part to their strong coaching staff, including Belichick, who has led the team to the playoffs in 17 of his 21 seasons as head coach. If you're interested in learning more about the Patriots, I'd be happy to provide you with more information about their current roster, upcoming games, and team statistics. Just let me know!
================================ Remove Message ================================


================================ Remove Message ================================


===========================

## Conferences:
- How to manage conversation history [link](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
- How to add summary of the conversation history [link text](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)